In [1]:
from numpy import *
from numpy import linalg as la

In [2]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]
    
def loadExData2():
    '''实际当中更稀疏的矩阵'''
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [3]:
'''
SVD分解测试
'''
Data = loadExData()
U, sigma, VT = la.svd(Data)
sigma

array([9.64365076e+00, 5.29150262e+00, 6.49628424e-16, 1.43063514e-16,
       2.79192092e-17])

In [4]:
'''
重构矩阵
'''
Sig2 = mat([[sigma[0], 0], [0, sigma[0]]])
recData = U[:, :2] * Sig2 * VT[:2, :]
recData

matrix([[ 4.14123564e-16, -9.03495868e-16, -9.03495868e-16,
          3.64495738e+00,  3.64495738e+00],
        [ 1.31761943e-15, -6.58809716e-16, -6.58809716e-16,
          5.46743607e+00,  5.46743607e+00],
        [ 4.39206478e-16, -2.19603239e-16, -2.19603239e-16,
          1.82247869e+00,  1.82247869e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         -3.10354619e-16, -3.10354619e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
         -4.47336215e-16, -4.47336215e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
         -1.01149745e-17, -1.01149745e-17],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         -2.23668107e-16, -2.23668107e-16]])

### 将11维的矩阵转换为一个3维的矩阵


In [15]:
Data = loadExData2()
U, sigma, VT = la.svd(Data)
energy = 0
Sig2 = sigma**2
thresh = sum(Sig2)*0.9
for i in range(len(sigma)):
    energy += Sig2[i]
    if energy > thresh:
        print(i,energy)
        break

2 500.5002891275793


# 基于协同过滤的推荐系统
将用户和其他用户的数据进行对比来实现推荐
计算两者之间的相似度，若相似度高则推荐

## 相似度计算
将其转换到0~1之间来进行计算
* 用户相似度：行与行之间的距离
* 物品相似度：列与列之间的距离

In [6]:
def ecludSim(inA,inB):
    '''
    欧氏距离
    '''
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    '''
    皮尔逊相关系数
    '''
    if len(inA) < 3 : 
        return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    '''
    余弦相似度
    '''
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [7]:
'''
测试
'''
myMat = mat(Data)
ecludSim(myMat[:,0],myMat[:,4])

0.08462632608958592

In [8]:
ecludSim(myMat[:,0],myMat[:,0])
#cosSim(myMat[:,0],myMat[:,4])
#cosSim(myMat[:,0],myMat[:,0])
#pearsSim(myMat[:,0],myMat[:,4])
#pearsSim(myMat[:,0],myMat[:,0])

1.0

# 菜馆菜肴推荐引擎
寻找没有吃过的菜, 通过SVD减少特征空间提高推荐效果
* 工作过程：给定一个用户，系统会为此用户返回N个最好的推荐菜
    1. 寻找用户没有评级的菜肴，即在 用户 - 物品 矩阵中的0值
    2. 在用户没有评级的所有物品中，对每一个物品预计一个可能的评级分数。即我们认为用户可能的打分
    3. 对这些物品的评分进行从大到小排序，返回前 N 个物品

In [9]:
def standEst(dataMat, user, simMeas, item):
    '''
    计算给定相似度计算方法的条件下，用户对物品的估计评分值
    输入：数据矩阵dataMat，用户编号user，相似度计算方法simMeans，物品编号item
    输出：估计评分值
    '''
    n = shape(dataMat)[1]                     # 得到数据中物品的数目
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):                        # 遍历数据中的每个物品
        userRating = dataMat[user,j]          # 得到用户的评分
        if userRating == 0:                   # 若评分值为0，则表明没人对其评分，跳过该物品   
            continue
        overLap = nonzero(logical_and(dataMat[:,item].A>0, dataMat[:,j].A>0))[0]  # 寻找两个用户都评级的物品
        
        if len(overLap) == 0:      # 若两者没有任何重叠物品，则相似度为0
            similarity = 0
        else:                      # 否则，基于重合的物品计算相似度
            similarity = simMeas(dataMat[overLap,item], dataMat[overLap,j])
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        
        simTotal += similarity     # 累计相似度的值
        ratSimTotal += similarity * userRating  #累计加权相似度的值
    if simTotal == 0: 
        return 0
    else: 
        return ratSimTotal/simTotal  # 归一化，使得评分在0-5之间

In [10]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    '''
    推荐引擎，用于产生最高的 N 个推荐结果
    输入：数据矩阵dataMat, 用户编号user，推荐数目 N，相似度计算方法simMeans， 估计方法estMethod
    输出：
    '''
    unratedItems = nonzero(dataMat[user,:].A==0)[1]         # 对给定的用户建立一个未评分的物品列表
    
    if len(unratedItems) == 0:                              # 如果列表为空，则说明都已经评分了，直接返回
        return 'you rated everything'                       
    
    itemScores = []                                         #  
    for item in unratedItems:                               # 遍历未评分列表中的物品 
        estimatedScore = estMethod(dataMat, user, simMeas, item)    # 对其进行预测评分
        itemScores.append((item, estimatedScore))                   # 将物品，及其预测的得分，添加到评分列表中
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]  #对 其进行从大到小排序，取前N个

In [11]:
'''测试'''
myMat = mat(loadExData())
myMat[0, 1] = myMat[0, 0] = myMat[1, 0] = myMat[2, 0] = 4
myMat[3, 3] = 2
print(myMat)

[[4 4 0 2 2]
 [4 0 0 3 3]
 [4 0 0 1 1]
 [1 1 1 2 0]
 [2 2 2 0 0]
 [5 5 5 0 0]
 [1 1 1 0 0]]


In [12]:
recommend(myMat, 2)  # 还可以用其他的相似度算法来进行推荐

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.928746
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0243290220056256)]

In [16]:
def svdEst(dataMat, user, simMeas, item):
    '''
    基于svd的评分估计
    输入：数据矩阵dataMat，用户编号user，相似度计算方法simMeans，物品编号item
    输出：估计评分值
    '''
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0 
    
    U,Sigma,VT = la.svd(dataMat)            # 对数据集进行SVD分解
    Sig4 = mat(eye(4)*Sigma[:4])            # 只利用包含 90% 能量的奇异值
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  # 将物品转换到低维空间
    
    for j in range(n):                      # 对与给定的用户，在他所属的物品上遍历           
        userRating = dataMat[user,j]        # 得到物品的评分 
        if userRating == 0 or j==item:      # 若指定物品的评分为 0 ，则跳过
            continue                        # 否则，两个人的相似度
        similarity = simMeas(xformedItems[item,:].T, xformedItems[j,:].T)
        
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: 
        return 0
    else: 
        return ratSimTotal/simTotal

In [17]:
recommend(myMat, 1, estMethod=svdEst)

the 1 and 0 similarity is: 0.498142
the 1 and 3 similarity is: 0.498131
the 1 and 4 similarity is: 0.509974
the 2 and 0 similarity is: 0.552670
the 2 and 3 similarity is: 0.552976
the 2 and 4 similarity is: 0.217301


[(2, 3.4177569186592387), (1, 3.3307171545585645)]

## 基于SVD的图像压缩

In [21]:
def printMat(inMat, thresh=0.8):
    '''
    打印矩阵
    '''
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k]) > thresh:
                print(1,end=' ')
            else: 
                print(0,end=' ')
        print('')

In [23]:
def imgCompress(numSV=3, thresh=0.8):
    '''
    对图像进行压缩
    输入：给定的奇异值数目numSV，阈值thresh
    '''
    myl = []                                             # 读入文件，将其写入一个矩阵中
    for line in open('data/0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
        
    myMat = mat(myl)
    print("****original matrix******")
    printMat(myMat, thresh)
    
    U,Sigma,VT = la.svd(myMat)                           # 对图像进行奇异值分解
    SigRecon = mat(zeros((numSV, numSV)))                 
    for k in range(numSV):                                
        SigRecon[k,k] = Sigma[k]                         # 对Sigma进行指定大小的 重构
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]         # 重构图片  
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

In [24]:
imgCompress(2)

****original matrix******
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 